<a href="https://colab.research.google.com/github/cindykhris/SummerInternship2020/blob/master/SummerInternship2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pine Biotech Summer Reasearch

**Cindy Pino**


Abstract: In this study, we will analyze the gene expression differnt coronaviruses (CoVs) infections to determine how each virus differs when causing a disease. Here, we focuses the analysis on samples from SARS-CoV-1, MERS, and SARS-CoV-2. 
The raw sequence data (fastq files) for the SARS-CoV-1 and MERS infections were downloaded from GEO [GSE56192](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE56192), including their corresponding Mock-treated controls). The raw sequence data (fastq files) for the SARS-CoV-2 infections were downloaded from GEO ([GSE147507](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE147507)), including their corresponding Mock-treated controls). 

In this Jupyter notebook, I will use both R and Python to analyze three viruses: MERS, SARS-CoV-1, amd SARS-CoV-2

##Using R and Python in the same Notebook
First, let's active R magic. Don't forget to use %%R before running a R code 


---

In [213]:
#active R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


##Import all the folders we will need for this files (R and Python)

###Python

In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

###R

In [ ]:
%%R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

R[write to console]: Bioconductor version 3.10 (BiocManager 1.30.10), R 3.6.3 (2020-02-29)

R[write to console]: Installing package(s) 'DESeq2'

R[write to console]: trying URL 'https://bioconductor.org/packages/3.10/bioc/src/contrib/DESeq2_1.26.0.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 2060061 bytes (2.0 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con

In [ ]:
%%R
library(DESeq2)


In [ ]:
%%R
install.packages('dplyr', lib = "/usr/lib/R/library")

In [ ]:
%%R
install.packages('EnhancedVolcano',lib = "/usr/lib/R/library")

In [ ]:
%%R
install.packages('ggplot',lib = "/usr/lib/R/library")


## SARS-CoV
Severe Acute Respiratory Syndrom Coronavirus (SARS-CoV) was first identified in 20013 (WHO). 

In [ ]:
%%R
df = read.table('drive/My Drive/SummerResearch/DESeq_SARS_expression_genes_FPKM.txt',skip = 1, header = TRUE)


In [ ]:
%%R
# Cleaning the file for processing
ColNames1 <- df$id #take the column name
df = df[,-1] #take the data part - numeric values
df=as.matrix(df)

In [ ]:
%%R
names(df) <- NULL

In [ ]:
%%R
#Remove NAs from dataset
datanew <- na.exclude(df)

In [ ]:
%%R
#Remove zeroes 
data <- filter(df, undersirable != 0)
head(data,10)

In [ ]:
%%R
#Basic Settings:

colors <- c(rep('red',6),rep('blue',3),rep('green',2),rep('gray',2))
par(mar=c(14,4,2,2))
boxplot(df, main="Gene Expression",las = 2, cex.axis=0.6)


In [ ]:

%%R
#Log transformation
logdata <- log(df+1)
par(mar=c(14,4,2,2))
boxplot(logdata, main="Log Transformed Gene Expression", col = 'blue', las = 2, cex.axis=0.6)


In [ ]:
%%R
#Descriptive statistics
summary(df)

In [ ]:
%%R
hist(df, col='pink')


In [ ]:
%%R
hist(logdata, col='magenta')

In [ ]:
%%R
barplot(sort(logdata[100,]), col = "blue", main = ColNames1[2], font.axis=1, cex.axis=1, las=2)


In [ ]:
%%R
heatmap(logdata)

###DESeq2 
Differential Gene Expression

In [ ]:
%%R
# Load the file - raw count
PDXdataset <- read.table("drive/My Drive/SummerResearch/DESeq_SARS_expression_genes.txt", header = TRUE, row.names=1, sep='\t')
#head(PDXdataset)

In [ ]:
%%R
#Create a dataframe - columns and rows
data <- (PDXdataset)
colnames(PDXdataset) -> names
meta_classes <- c('MK','MK','MK','MK','MK','MK','SA','SA','SA','SA','SA','SA')
meta_data <- data.frame(names, meta_classes)
head(data)

In [ ]:
%%R
#Construct DESeq2 dataset object and filter the zeroes
ds <- DESeqDataSetFromMatrix(countData=data, colData=meta_data, design=~meta_classes)




In [ ]:
%%R

deseq2Data <- DESeq(ds)